<a href="https://colab.research.google.com/github/viviandonohoe/machinelearningproject/blob/main/ml_spanishnotarialdocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code I'm working on for optimizing OCR on Spanish notarial documents.
